In [1]:
#Importing libraries
from pinecone import Pinecone, ServerlessSpec

In [2]:
#Initializing pinecone client
pc = Pinecone(api_key="dd1385d4-5e6c-4002-9db4-d0f08bd2b1d0")

In [3]:
#Create an index
index_name = "embeddings"

#Check if the index already exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name = index_name,
        metric="cosine",
        dimension=768,
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        )
    )

#Get the index
index = pc.Index(index_name)

In [4]:
#Loading all the embeddings
import numpy as np

sentence_embeddings = np.load("D:/My projects/RAG-based Semantic Search/Data/handling_data/text_data/sentence_embeddings.npy")
sparse_embeddings = np.load("D:/My projects/RAG-based Semantic Search/Data/handling_data/text_data/sparse_embeds.npy", allow_pickle=True)
image_embeddings = np.load("D:/My projects/RAG-based Semantic Search/Data/handling_data/image_data/image_embeds.npy").squeeze(1)
#importing json as dictionary 
import json

with open("D:\My projects\RAG-based Semantic Search\Data\metadata\metadata.json", "r") as f:
    metadata = json.load(f)

<>:10: SyntaxWarning: invalid escape sequence '\M'
<>:10: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Dell\AppData\Local\Temp\ipykernel_16832\590403953.py:10: SyntaxWarning: invalid escape sequence '\M'
  with open("D:\My projects\RAG-based Semantic Search\Data\metadata\metadata.json", "r") as f:


In [5]:
#Upsert just the text embeddings
for id, (dense_sent, sparse_sent, img, meta) in enumerate(zip(sentence_embeddings, sparse_embeddings, image_embeddings, metadata)):
    index.upsert(namespace='embeds', vectors=[
        {
            "id": str(id),
            "values" : dense_sent.tolist(),
            "sparse_values" : {
                "indices" : sparse_sent['indices'],
                "values" : sparse_sent['values']
            },
            "metadata" :{
                'name': meta['name'],
                'price': meta['price'],
                'color': meta['color'],
                'category': meta['category'],
                'description': meta['description'],
                'image_path': meta['image_path']
            }
        }
    ])

In [6]:
#Creating the text + image embeddings

tot_embeddings = list()

for i in range(0,843):

    tot_embeddings.append(sentence_embeddings[i].tolist() + image_embeddings[i].tolist())

In [7]:
#Creating a different index
index_name = "totembeddings"

# #Check if the index already exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name = index_name,
        metric="cosine",
        dimension=1280,
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        )
    )

#Get the index
index = pc.Index(index_name)

In [8]:
#upserting text+image embeddinsg in a difference namespacce

for id, (total_embedding, sparse_sent, meta) in enumerate(zip(tot_embeddings, sparse_embeddings, metadata)):
    index.upsert(namespace='total_embeds', vectors=[
        {
            "id": str(id),
            "values" : total_embedding,
            "sparse_values" : {
                "indices" : sparse_sent['indices'],
                "values" : sparse_sent['values']
            },
            "metadata" :{
                'name': meta['name'],
                'price': meta['price'],
                'color': meta['color'],
                'category': meta['category'],
                'description': meta['description'],
                'image_path': meta['image_path']
            }
        }
    ])